In [1]:
# So get a list of players at the start of a quarter for each time (home 1-5 and away 1-5)
# I think I'll need to loop through each row of the dataframe and modify the list of players for each sub event (home and away)
#     Will also calculate the margin, score, and seconds for each

In [12]:
import requests
import pandas as pd
import numpy as np
import random
import time
from sklearn.preprocessing import MultiLabelBinarizer
pd.set_option('display.max_columns',500)

In [3]:
# play_by_play_url = 'https://stats.nba.com/stats/playbyplayv2/?gameId=0041700404&&startPeriod=0&endPeriod=14' #
# games_url = 'https://stats.nba.com/stats/leaguegamelog?\
# Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00\
# &PlayerOrTeam=T&Season=2021-22&SeasonType=Playoffs&Sorter=DATE' #
# players_game_url = 'https://stats.nba.com/stats/boxscoreadvancedv2/?gameId=0041700404&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2'

In [4]:
# Headers for querying NBA JSON data api
header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [5]:
# code for getting starting quarter lineups from here: https://github.com/rd11490/NBA-Play-By-Play-Example
# endpoints
def play_by_play_url(game_id):
    return "https://stats.nba.com/stats/playbyplayv2/?gameId={0}&startPeriod=0&endPeriod=14".format(game_id)

def advanced_boxscore_url(game_id, start, end):
    return "https://stats.nba.com/stats/boxscoreadvancedv2/?gameId={0}&startPeriod=0&endPeriod=14&startRange={1}&endRange={2}&rangeType=2".format(game_id, start, end)

def games_url(season,season_type):
    return 'https://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season={}&SeasonType={}&Sorter=DATE'.format(season,season_type)

def extract_data(url):
    resp = requests.get(url, headers=header_data).json()
#     resp = json.loads(r.data)
    results = resp['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    frame = pd.DataFrame(rows)
    frame.columns = headers
    return frame

def calculate_time_at_period(period):
    if period > 5:
        return (720 * 4 + (period - 5) * (5 * 60)) * 10
    else:
        return (720 * (period - 1)) * 10
    
def split_subs(df, tag):
    subs = df[[tag, 'PERIOD', 'EVENTNUM']]
    subs['SUB'] = tag
    subs.columns = ['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']
    return subs

def get_all_games(season,season_type):
    games = extract_data(games_url(season,season_type))
    games = games[games['MATCHUP'].str.contains('vs.')][['GAME_ID','MATCHUP']]
    games['HOME']=games['MATCHUP'].str.split(' vs. ').str.get(0)
    games['AWAY']=games['MATCHUP'].str.split(' vs. ').str.get(1)
    return games


def get_game_data(game_id,games):
    
    frame = extract_data(play_by_play_url(game_id)).fillna('')
    
    substitutionsOnly = frame[frame["EVENTMSGTYPE"] == 8][['PERIOD', 'EVENTNUM', 'PLAYER1_ID', 'PLAYER2_ID']]
    substitutionsOnly.columns = ['PERIOD', 'EVENTNUM', 'OUT', 'IN']

    subs_in = split_subs(substitutionsOnly, 'IN')
    subs_out = split_subs(substitutionsOnly, 'OUT')

    full_subs = pd.concat([subs_out, subs_in], axis=0).reset_index()[['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']]
    first_event_of_period = full_subs.loc[full_subs.groupby(by=['PERIOD', 'PLAYER_ID'])['EVENTNUM'].idxmin()]
    players_subbed_in_at_each_period = first_event_of_period[first_event_of_period['SUB'] == 'IN'][['PLAYER_ID', 'PERIOD', 'SUB']]

    periods = players_subbed_in_at_each_period['PERIOD'].drop_duplicates().values.tolist()

    frames = []
    for period in periods:

        low = calculate_time_at_period(period) + 5
        high = calculate_time_at_period(period + 1) - 5
        boxscore = advanced_boxscore_url(game_id, low, high)
        boxscore_players = extract_data(boxscore)[['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION']]
        boxscore_players['PERIOD'] = period

        players_subbed_in_at_period = players_subbed_in_at_each_period[players_subbed_in_at_each_period['PERIOD'] == period]

        joined_players = pd.merge(boxscore_players, players_subbed_in_at_period, on=['PLAYER_ID', 'PERIOD'], how='left')
        joined_players = joined_players[pd.isnull(joined_players['SUB'])][['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION', 'PERIOD']]
        frames.append(joined_players)

    out = pd.concat(frames)    
    out = out.groupby(['TEAM_ABBREVIATION','PERIOD'])['PLAYER_ID'].apply(lambda x: ','.join(map(str, x))).reset_index()
    
    frame['HOME']=games[games['GAME_ID']==game_id]['HOME'].values[0]
    frame['AWAY']=games[games['GAME_ID']==game_id]['AWAY'].values[0]
    frame = frame.merge(out,how='left',left_on=['HOME','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
    frame.rename({'PLAYER_ID':'HOME_PLAYER_IDS'},axis=1,inplace=2)
    frame = frame.merge(out,how='left',left_on=['AWAY','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
    frame.rename({'PLAYER_ID':'AWAY_PLAYER_IDS'},axis=1,inplace=2)
    frame['HOME_SUB']=frame['HOMEDESCRIPTION'].str.contains('SUB')
    frame['AWAY_SUB']=frame['VISITORDESCRIPTION'].str.contains('SUB')
    frame['QUARTER_START']=frame['NEUTRALDESCRIPTION'].str.contains('Start')
    frame['PLAYER_CHANGE']=np.where((frame['QUARTER_START']==True)|(frame['AWAY_SUB']==True)|(frame['HOME_SUB']==True),True,False)
    frame['HOME_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['HOME_PLAYER_IDS'],np.nan)
    frame['AWAY_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['AWAY_PLAYER_IDS'],np.nan)
    
    for idx, row in frame.iterrows():
        if idx == 0 or row['QUARTER_START']==True:
            continue
        else:
            frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx-1,'AWAY_PLAYER_IDS']
            frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx-1,'HOME_PLAYER_IDS']
            if row['AWAY_SUB']==True:
                frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx,'AWAY_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
            if row['HOME_SUB']==True:
                frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx,'HOME_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
    frame['HOME_PLAYER_IDS'].ffill(inplace=True)
    frame['AWAY_PLAYER_IDS'].ffill(inplace=True)
    
    frame=frame[frame['PLAYER_CHANGE']==False]
    frame=frame[frame['NEUTRALDESCRIPTION']=='']
    frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Jump Ball'))&(~frame['VISITORDESCRIPTION'].str.contains('Jump Ball'))]
    frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Timeout'))&(~frame['VISITORDESCRIPTION'].str.contains('Timeout'))]
    
    frame['NEXT_HOME_PLAY']=frame['HOMEDESCRIPTION'].shift(-1)
    frame['NEXT_AWAY_PLAY']=frame['VISITORDESCRIPTION'].shift(-1)
    frame['TIMER']=frame['PCTIMESTRING'].str.split(':').str.get(0).astype(int)+frame['PCTIMESTRING'].str.split(':').str.get(1).astype(int)/60
    frame['SECONDS_PASSED']=np.where(frame['TIMER'].shift(1)>=frame['TIMER'],frame['TIMER'].shift(1)*60-frame['TIMER']*60,(np.ceil(frame['TIMER'])-frame['TIMER'])*60)
    frame['HOME_POSSESSION_FLAG']=np.where((frame['NEXT_HOME_PLAY']=='')&(frame['HOMEDESCRIPTION']!=''),1,0)
    frame['AWAY_POSSESSION_FLAG']=np.where((frame['NEXT_AWAY_PLAY']=='')&(frame['VISITORDESCRIPTION']!=''),1,0)
    
    frame['AWAY_SCORE']=frame['SCORE'].str.split(' - ').str.get(0)
    frame['AWAY_SCORE']=np.where(frame['AWAY_SCORE']=='',np.nan,frame['AWAY_SCORE'])
    frame['AWAY_SCORE'].ffill(inplace=True)
    frame['AWAY_SCORE'].fillna(0,inplace=True)
    frame['HOME_SCORE']=frame['SCORE'].str.split(' - ').str.get(1)
    frame['HOME_SCORE'].ffill(inplace=True)
    frame['HOME_SCORE'].fillna(0,inplace=True)
    frame['HOME_SCORE_DIFFERENCE']=frame['HOME_SCORE'].astype(int)-frame['HOME_SCORE'].astype(int).shift(1)
    frame['AWAY_SCORE_DIFFERENCE']=frame['AWAY_SCORE'].astype(int)-frame['AWAY_SCORE'].astype(int).shift(1)
    frame['HOME_SCORE_DIFFERENCE'].fillna(0,inplace=True)
    frame['AWAY_SCORE_DIFFERENCE'].fillna(0,inplace=True)
    frame['HOME_VS_AWAY']=frame['HOME_SCORE_DIFFERENCE']-frame['AWAY_SCORE_DIFFERENCE']
    return frame

In [11]:
season = '2021-22'
season_type = 'Regular+Season'
games = get_all_games(season,season_type)
games

,GAME_ID,MATCHUP,HOME,AWAY
1,0022101224,MIN vs. CHI,MIN,CHI
2,0022101228,PHI vs. DET,PHI,DET
5,0022101223,MEM vs. BOS,MEM,BOS
7,0022101220,DEN vs. LAL,DEN,LAL
8,0022101227,ORL vs. MIA,ORL,MIA
...,...,...,...,...
2450,0022100006,TOR vs. WAS,TOR,WAS
2452,0022100004,DET vs. CHI,DET,CHI
2454,0022100003,CHA vs. IND,CHA,IND
2457,0022100001,MIL vs. BKN,MIL,BKN


In [ ]:
# # Run the dataframe all together
# for i,game in enumerate(games['GAME_ID'].values[0:10]):
#     if i == 0:
#         results = get_game_data(game_id=game,games=games)
#     else:
#         results = pd.concat([results,(get_game_data(game_id=game,games=games))],axis=1)
# results

In [43]:
now_time = str(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())).replace(' ','-').replace(':','-')
problem_games = []
completed_games = []
for i,game in enumerate(games['GAME_ID'].values):
    if i == 0:
        results = get_game_data(game_id=game,games=games)
        results.to_csv('play_by_play_run'+now_time+'.csv')
        completed_games.append(game)
    else:
        try:
            results = get_game_data(game_id=game,games=games)
            completed_runs.append(game)
            results.to_csv('play_by_play_run'+now_time+'.csv', mode='a', header=False)
        except Exception as e:
            print('Problem with run ' + str(i+1) + ', game id ' + str(game))
            print(e)
            problem_games.append(game)
        if (i+1)%10 == 0:
            print('Run ' + str(i+1) + ' completed. ' + str(len(games['GAME_ID'])-i+1) + ' to go.')
    # Wait time to not get banned from API
    time.sleep(random.randint(5,10))
print('Done with the errors in the problem_games list.')

Run 10 completed. 1222 to go.
Problem with run 12, game id 0022101230
HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/boxscoreadvancedv2/?gameId=0022101230&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Problem with run 13, game id 0022101225
HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/playbyplayv2/?gameId=0022101225&startPeriod=0&endPeriod=14 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Problem with run 15, game id 0022101222
HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/playbyplayv2/?gameId=0022101222&startPeriod=0&endPeriod=14 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Run 20 completed. 1212 to go.
Problem with run 27, game id 0022101203
HTTP

KeyboardInterrupt: 

In [70]:
test_df = pd.read_csv('play_by_play_run'+now_time+'.csv')
mlb = MultiLabelBinarizer()
test_df['HOME_PLAYER_IDS']=test_df['HOME_PLAYER_IDS'].str.split(',')
test_df['AWAY_PLAYER_IDS']=test_df['AWAY_PLAYER_IDS'].str.split(',')
test_df = test_df.merge(pd.DataFrame(mlb.fit_transform(test_df['HOME_PLAYER_IDS']),columns=mlb.classes_, index=test_df.index),
                     how='inner', left_index=True, right_index=True)
# test_df = test_df.merge(pd.DataFrame(mlb.fit_transform(test_df['AWAY_PLAYER_IDS']),columns=mlb.classes_, index=test_df.index),
#                      how='inner', left_index=True, right_index=True)
for idx,row in test_df.iterrows():
    for player in row['AWAY_PLAYER_IDS']:
        test_df.at[idx,player]=-1
test_df.iloc[:,55:].fillna(0,inplace=True)
test_df.to_csv('test_df'+now_time+'.csv')
test_df

,Unnamed: 0,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,HOME,AWAY,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,HOME_SUB,AWAY_SUB,QUARTER_START,PLAYER_CHANGE,NEXT_HOME_PLAY,NEXT_AWAY_PLAY,TIMER,SECONDS_PASSED,HOME_POSSESSION_FLAG,AWAY_POSSESSION_FLAG,AWAY_SCORE,HOME_SCORE,HOME_SCORE_DIFFERENCE,AWAY_SCORE_DIFFERENCE,HOME_VS_AWAY,1626145,1626147,1626149,1626153,1626156,1626157,1626162,1626168,1626169,1626171,1626172,1626179,1626181,1626192,1626195,1626196,1626204,1626220,1626224,1626246,1627736,1627737,1627741,1627745,1627747,1627751,1627752,1627759,1627774,1627777,1627780,1627783,1627788,1627789,1627814,1627823,1627826,1627827,1627832,1627853,1627854,1627863,1627884,1627936,1628021,1628365,1628367,1628369,1628370,1628373,1628374,1628378,1628379,1628380,1628389,1628392,1628396,1628401,1628402,1628415,1628418,1628420,1628425,1628427,1628432,1628436,1628449,1628464,1628470,1628539,1628960,1628963,1628964,1628966,1628969,1628970,1628971,1628972,1628973,1628975,1628978,1628981,1628984,1628988,1628989,1628991,1628995,1628997,1628998,1629001,1629002,1629003,1629006,1629011,1629013,1629018,1629020,1629021,1629022,1629023,1629027,1629029,1629033,1629035,1629048,1629052,1629060,1629083,1629109,1629111,1629117,1629130,1629139,1629162,1629164,1629216,1629308,1629312,1629599,1629607,1629611,1629622,1629628,1629630,1629631,1629632,1629633,1629634,1629636,1629637,1629639,1629640,1629641,1629643,1629645,1629649,1629650,1629651,1629655,1629658,1629659,1629661,1629667,1629669,1629673,1629675,1629676,1629678,1629680,1629684,1629685,1629717,1629719,1629723,1629726,1629750,1629752,1629875,1629962,1630162,1630163,1630165,1630166,1630167,1630168,1630169,1630170,1630171,1630172,1630173,1630174,1630175,1630176,1630177,1630178,1630180,1630181,1630182,1630183,1630188,1630191,1630192,1630193,1630194,1630195,1630198,1630200,1630201,1630202,1630205,1630208,1630209,1630210,1630214,1630215,1630216,1630217,1630219,1630222,1630224,1630225,1630227,1630228,1630230,1630231,1630233,1630238,1630240,1630245,1630249,1630256,1630264,1630267,1630285,1630296,1630322,1630526,1630528,1630529,1630530,1630532,1630533,1630536,1630538,1630539,1630541,1630543,1630547,1630550,1630551,1630552,1630556,1630557,1630558,1630559,1630560,1630561,1630563,1630567,1630573,1630578,1630579,1630580,1630582,1630583,1630586,1630587,1630589,1630591,1630593,1630595,1630596,1630598,1630606,1630612,1630613,1630624,1630625,1630631,1630648,1630678,1630686,1630688,1630691,1630792,1630846,1630994,200752,200765,200768,200782,201142,201143,201144,201145,201152,201566,201567,201568,201571,201572,201577,201580,201587,201588,201599,201935,201942,201950,201959,201961,201976,201980,201988,202066,202083,202326,202328,202331,202334,202340,202355,202357,202362,202397,202681,202684,202685,202687,202691,202692,202694,202696,202699,202704,202709,202710,202711,202722,202738,203076,203082,203083,203084,203085,203087,203095,203107,203110,203114,203115,203200,203210,203468,203473,203476,203482,203484,203486,203490,203493,203496,203497,203500,203503,203504,203507,203526,203552,203897,203901,203903,203915,203918,203920,203932,203935,203937,203939,203952,203954,203991,203992,203999,204001,204456,2544,2546,2730,2738,1629076,1630568,203585,1630271,1630346,1630644,203506,2617,1628384,1629004,1628221,1629670,203648,1630572,1629623,1629730,1629674,1627749,1628982,203109,1629028,1626164,101108,1629604,1630535,1629234,1630553,1630314,203463,1629833,1629712,1629617,1630190,1630540,203145,1627739,1626166,101150,1629656,1627742,1629683,202693,202330,203458,1626208,1628